In [1]:
from bs4 import BeautifulSoup
import geopandas as gpd
import requests
import pandas as pd

## Open GIS Data

In [2]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)
states_df = states_df.to_crs(4326)

## Retrieve data

In [7]:
url = "https://caredge.com/guides/cheapest-state-to-buy-a-car"

In [8]:
r = requests.get(url)

In [9]:
soup = BeautifulSoup(r.text, "html.parser")

In [10]:
tbody = soup.find("tbody")
trs = tbody.find_all("tr")
state_dicts = []
for tr in trs:
    tds = tr.find_all("td")
    registration_fee = int(tds[-2].text.replace("$", "").replace(",", ""))
    state_dict = {"NAME": tds[0].text, "REGISTRATION_FEE": registration_fee}
    state_dicts.append(state_dict)

In [11]:
car_price_df = pd.DataFrame(state_dicts)

## Merge data

In [12]:
vehicle_gdf = states_df[["NAME", "geometry"]].merge(
    car_price_df, on="NAME", how="inner"
)

## Export

In [13]:
vehicle_gdf = vehicle_gdf.to_crs(9311)

In [14]:
vehicle_gdf.to_file("data/vehicle.gpkg")